In [2]:
# add current directory to path
import sys
sys.path.insert(0, '..')

import pycwb

print("Version used for this test: %s" % pycwb.__version__)

Version used for this test: 0.28.1.dev4+g3319959.d20250228


In [ ]:
from pycwb.config import Config
from pycwb.modules.logger import logger_init
from pycwb.modules.injection.par_generator import get_injection_list_from_parameters, repeat
from pycwb.modules.injection.sky_distribution import generate_sky_distribution, distribute_injections_on_sky
from pycwb.modules.injection.injection import distribute_inj_in_gps_time
from math import ceil
import numpy as np

logger_init()

injection = {
    'rate': '1/200',
    'jitter': 50,
    'allow_reuse_data': True,
    'repeat_injection': 10,
    'sky_distribution': {
        'type': 'UniformAllSky'
    },
    'parameters_from_python': {
        'file': 'injection_parameters.py',
        'function': 'get_injection_parameters'
    },
    'approximant': 'Ringdown',
    'generator': {
        'module': './input/waveform.py', 
        'function': 'get_td_waveform'
    }
}

repeat_injection = injection['repeat_injection']
rate = eval(injection['rate'])
jitter = injection['jitter']
sky_distribution = injection['sky_distribution']

print(f"Repeat injection: {repeat_injection}")
print(f"Rate: {rate}")
print(f"Jitter: {jitter}")
print(f"Sky distribution: {sky_distribution}")

total_live_time = 1000000
start_gps_time = 1262304000
end_gps_time = start_gps_time + total_live_time
injections = get_injection_list_from_parameters(injection)
print(f"Number of injections: {len(injections)}")
injections = repeat(injections, repeat_injection)
print(f"Number of injections after repeat: {len(injections)}")

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sky_distribution = {'type': 'UniformAllSky'}

sky_locations = generate_sky_distribution(sky_distribution, len(injections))
print(f"Number of sky locations: {len(sky_locations[0])}")
distribute_injections_on_sky(injections, sky_locations)
print(f"Number of injections after sky distribution: {len(injections)}")

ra = [inj['ra'] for inj in injections]
dec = [inj['dec'] for inj in injections]
# plot sky distribution
plt.figure(figsize=(4, 2))
plt.scatter(ra, dec, s=1)
plt.xlabel('RA')
plt.ylabel('DEC')
plt.show()

## Patch sky distribution

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

sky_distribution = {
    'type': 'Patch',
    'patch': {
        'center': [90, 90],
        'radius': 10
    }
}

sky_locations = generate_sky_distribution(sky_distribution, len(injections))
print(f"Number of sky locations: {len(sky_locations[0])}")
distribute_injections_on_sky(injections, sky_locations)
print(f"Number of injections after sky distribution: {len(injections)}")

ra = np.array([inj['ra'] for inj in injections])
dec = np.array([inj['dec'] for inj in injections])
# plot sky distribution
plt.figure(figsize=(4, 4))
plt.scatter(ra, dec, s=1)
plt.xlabel('RA')
plt.ylabel('DEC')
plt.show()

## Distribute in time

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

distribute_inj_in_gps_time(injections, rate, jitter, start_gps_time, end_gps_time, shuffle=False)

plt.figure(figsize=(4, 4))
plt.plot([inj['gps_time'] for inj in injections], [inj['trail_idx'] for inj in injections], 'o')
plt.xlim(start_gps_time, end_gps_time)
plt.show()

In [ ]:
injections[0]